In [ ]:
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import pickle

In [ ]:
#kvk-nummers die niet werken
if os.path.isfile("kvk_companyinfo_skip" + ".p"):
    with open("kvk_companyinfo_skip" + ".p", 'rb') as fp:
        skip = pickle.load(fp)
else: skip = []

In [ ]:
#processed kvk-nummers (in het geval van een restart van de run om duplicates the voorkomen)
if os.path.isfile("kvk_companyinfo.csv"):
    kvk_processed = pd.read_csv("kvk_companyinfo.csv",dtype=object)
else:
    columns = ["kvknr","reg_date","emp_class"]
    kvk_processed = pd.DataFrame(columns=columns)

In [ ]:
#extract de informatie van companyinfo op basis van het kvk-nummer
def extract(kvknr):
    global skip
    
    #zoek functie om het bedrijf te vinden van het kvk-nummer
    url = "https://companyinfo.nl/freemium/?externalSearch="+kvknr
    html = request.urlopen(url).read().decode('utf8')
    raw = BeautifulSoup(html, 'html.parser').get_text()
    
    #vind de link naar het bedrijf en extract de informatie    
    if raw.find('seoUrl') > 0:
        new_url = ('/').join(raw[raw.find('seoUrl')+15:raw.find('resultAmount')-5].split('\\u002F'))
        prefix = "https://companyinfo.nl/organisatieprofiel/"
        new_html = request.urlopen(prefix+new_url).read().decode('utf8')
        new_raw = BeautifulSoup(new_html, 'html.parser').get_text()

        reg_date = new_raw[new_raw.find('registrationDate')+19:new_raw.find('registrationDate')+29]
        emp_class = new_raw[new_raw.find('employeeClass')+16:new_raw.find('employeeClass')+18]
        return reg_date, emp_class
    else:
        skip.append(kvknr)
        return None, None

In [ ]:
#voorbeeld
kvknr = '01036591'
reg_date, emp_class = extract(kvknr)

print('registraion date:\t', reg_date)
print('employee class:\t\t', emp_class)

In [ ]:
#dataset net kvk-nummers uit de kvk-dataset
kvknr_dict = pd.read_csv("../kvk/kvknr.csv", header=None, dtype=str)
kvknr_dict.columns = ["kvknr"]

In [ ]:
#loop over alle kvk-nummers en extract de informatie van companyinfo
#kvk-nummers in skip geven geen resultaten bij companyinfo
#sleep is toegevoegd om de site niet te overloaden
for kvknr in kvknr_dict["kvknr"]:
    if kvknr not in kvk_processed["kvknr"].to_list():
        if kvknr not in skip:
            print(kvknr)
            time.sleep(5)
            reg_date, emp_class = extract(kvknr)
            kvk_processed = kvk_processed.append(pd.DataFrame([[kvknr,reg_date,emp_class]],columns=columns),ignore_index=True)

In [ ]:
kvk_processed.tail()

In [ ]:
#opslaan van resultaten
kvk_processed.to_csv("kvk_companyinfo.csv",index=False)

In [ ]:
#opslaan van kvk-nummers zonder resultaat
with open("kvk_companyinfo_skip" + '.p', 'wb') as fp:
        pickle.dump(skip, fp)